<a href="https://colab.research.google.com/github/handsome1201/LgAimers/blob/main/LG_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('/content/drive/MyDrive/LGAI/train.csv')

In [3]:
train_data = pd.read_csv('/content/drive/MyDrive/LGAI/train.csv').drop(columns=['제품','대분류','중분류','소분류','브랜드'])

In [4]:
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train_data.values.reshape(-1, 1))

In [5]:
sequence_length = 21  # 시퀀스 길이 (과거 일수)
X_train = []
y_train = []
for i in range(len(scaled_train) - sequence_length):
    X_train.append(scaled_train[i:i+sequence_length])
    y_train.append(scaled_train[i+sequence_length])
X_train = np.array(X_train)
y_train = np.array(y_train)

# LSTM 모델 생성
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [6]:
model.fit(X_train, y_train, epochs=5, batch_size=32)

# 예측을 위한 test 데이터 생성
last_sequence = scaled_train[-sequence_length:]
X_test = []
for _ in range(21):
    X_test.append(last_sequence[-sequence_length:])
    predicted_value = model.predict(np.array([last_sequence]))
    last_sequence = np.concatenate((last_sequence[1:], predicted_value), axis=0)
X_test = np.array(X_test)

# 예측 수행
predicted_scaled = model.predict(X_test)
predicted_sales = scaler.inverse_transform(predicted_scaled)


Epoch 1/5
228419/228419 [==============================] - 2428s 11ms/step - loss: 6.1109e-06
Epoch 2/5
228419/228419 [==============================] - 2345s 10ms/step - loss: 6.0674e-06
Epoch 3/5
228419/228419 [==============================] - 2346s 10ms/step - loss: 5.8367e-06
Epoch 4/5
228419/228419 [==============================] - 2321s 10ms/step - loss: 5.6213e-06
Epoch 5/5
1/1 [==============================] - 0s 26ms/step


In [8]:
# 예측 결과를 DataFrame으로 변환
predicted_df = pd.DataFrame({'Predicted_Sales': predicted_sales.reshape(-1)})

# 예측 결과를 CSV 파일로 저장
predicted_df.to_csv('predicted_results.csv', index=False)

In [9]:
submit = pd.read_csv('/content/drive/MyDrive/LGAI/sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
submit.iloc[:,1:] = pred
submit.head()